##### INSTALAÇÃO DOS PACOTES

In [ ]:
# Instalando os pacotes
# Executar o seguinte comando no console: 
#!pip install -r requirements.txt
#!pip3 install -U googlemaps
#!pip3 install folium
#!pip3 install ortools

##### IMPORTANDO AS BIBLIOTECAS

In [ ]:
import requests
import googlemaps
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import folium
import json

import warnings
warnings.filterwarnings('ignore')
# -*- coding: utf-8 -*-

##### INICIANDO API GOOGLE MAPS E CRIANDO FUNCAO PARA PEGAR COORDENADAS COM GEOCODE

In [ ]:
# Iniciando API google maps

api_key = json.load(open(r'C:\Users\Valeria\Projetos_DS\cred_gclp.json')).get('api_key')
gmaps = googlemaps.Client(key=api_key)

In [ ]:
# Função para obter informações de latitude e longitude
def coordenadas(cidade):
    geocode = gmaps.geocode(cidade)
    lat = geocode[0]['geometry']['location']['lat']
    lng = geocode[0]['geometry']['location']['lng']
    
    return [lat, lng]

##### IMPORTANDO DADOS

In [ ]:
# Importando dados
CDs = pd.read_csv('./bases/cidades_cds.csv', sep=',',encoding='utf-8')
CDs['CITY - UF'] = CDs['CITY'] + ' - ' + CDs['UF']
CDs.drop(columns=['CITY','UF'], inplace=True)

cidades_iniciais = CDs['CITY - UF'].unique()

In [ ]:
# Coletando Latitude e Longitude via API Google Maps
for linha in range(0,len(CDs)):
    coord = coordenadas(CDs.loc[linha, 'CITY - UF'])
    CDs.loc[linha, 'LAT'] = coord[0]
    CDs.loc[linha, 'LNG'] = coord[1]
    
del coord

In [ ]:
# Criando array com coordenadas das cidades
coordenadas = []
for linha in range(0,len(CDs)):
    coordenada = [CDs.loc[linha, 'LAT'],CDs.loc[linha, 'LNG']]
    coordenadas.append(coordenada)
    
    cidades = np.array(coordenadas)
    
del coordenada, coordenadas

In [ ]:
# Carregando poligonos cidades UF SP. Repositorio fonte: https://github.com/tbrugz/geodata-br/tree/master
mun_sp = json.load(open('./bases/geojs-35-mun.json', encoding='utf-8'))

In [ ]:
# Função para plotar mapa usando pacote Folium

#Cores disponiveis: {'lightred','red','darkred','lightblue','blue','darkblue','cadetblue','lightgreen','green','darkgreen',
#'white','lightgray','gray','black','pink','purple','darkpurple','beige','orange'}

def plotar_mapa(info, base, cidades, poligonos, cores_clusters):
    
    # Criando mapa centralizado no estado de SP (parametro location)
    mapa = folium.Map(width=800, height=500, location = [-22.9050,-47.06133], zoom_start = 6)                                            
    folium.GeoJson(mun_sp,style_function=lambda feature: {'fillColor': 'gray','color': 'darkred','weight': 0.8,}).add_to(mapa)
    
    # Se info = 'inicial', plota mapa inicial
    if info == 'inicial':
        
        #Adicionando marcadores para cada coordenada
        for item in range(0,len(cidades)):
            folium.Marker(cidades[item], icon=folium.Icon(color = 'orange')).add_to(mapa)
        return mapa
    
    # Se info = 'cluster', plota mapa clusterizado
    elif info == 'cluster':
        
        # Criando um dicionário de cores para os clusters
        cores = {0: 'blue', 1: 'red', 2: 'green', 3: 'pink'}

        # Adicionando marcadores para cada coordenada com cores correspondentes aos clusters
        for coord, cluster in zip(cidades, cores_clusters):
            folium.Marker(location=coord, icon=folium.Icon(color=cores[cluster])).add_to(mapa)

        # Adicionando marcadores para os centroides
        for item in range(0,len(base)):
            folium.Marker(base[item],icon=folium.Icon(color = 'black')).add_to(mapa)

        # Adicionando circunferencias de raio 150km, considerando premissa do problema
        for item in range(0,len(base)):
            folium.Circle(base[item], radius = 150000, color='black', weight=1, fill=True, fill_color='beige', fill_opacity=0.2).add_to(mapa)
    
        return mapa

In [ ]:
# Plotando mapa inicial para visualização
mapa = plotar_mapa('inicial', 0, cidades, mun_sp, 0)
mapa

In [ ]:
# Aplicação do algoritmo k-means

# Quantidade de clusters desejado: escolhido valor 4 por haver apenas 4 gerentes para SP
k = 4  

# Criando o objeto K-Means
kmeans = KMeans(n_clusters=k, init='random', n_init='auto').fit(cidades)

# Obtendo os rótulos dos clusters para as cidades
rotulos_clusters = kmeans.labels_

# Obtendo os centros finais após a clusterização
clusters = kmeans.cluster_centers_

In [ ]:
# Plotando mapa clusterizado com centroides
mapa2 = plotar_mapa('cluster', clusters, cidades, mun_sp, rotulos_clusters)
mapa2

##### TSP

In [ ]:
# TSP - Travelling Sallesman Problem (Problema do Caixeiro Viajante)

CDs['CLUSTERS'] = rotulos_clusters
CDs[['BASES','EXISTE']] = [0,1]

In [ ]:
def coletando_dados_centroides(lat,lng):
    url = "https://maps.googleapis.com/maps/api/geocode/json?"
    coord = str(lat) + ',' + str(lng)

    # Fazendo solicitação HTTP
    r = requests.get(url + "latlng=" + coord + "&key="+api_key).json()
    
    # Filtrando dados
    r = r['results'][0]['address_components']
    
    # Loop buscando a chave
    for item in r:
        if 'administrative_area_level_2' in item.get('types', []):
            cidade = str(item['short_name']).upper()
        if 'administrative_area_level_1' in item.get('types', []):
            uf = str(item['short_name']).upper()
            
    cid_centro = cidade + ' - ' + uf
    
    return [lat, lng, cid_centro]

In [ ]:
dados = []
for cluster in clusters:
    resp = coletando_dados_centroides(round(cluster[0],6),round(cluster[1],6))
    dados.append(resp)
    
base_centros = pd.DataFrame(dados, columns=['LAT','LNG','CITY - UF'])
base_centros['CLUSTERS'] = base_centros.index
base_centros[['BASES','EXISTE']] = [1,0]

for index, row in base_centros.iterrows():
    x = row['CITY - UF'] 
    if x in cidades_iniciais:
        base_centros.loc[index,'EXISTE'] = 1
        
CDs = pd.concat([CDs,base_centros], axis=0, join='inner')
CDs.drop_duplicates(subset='CITY - UF', keep='last', inplace=True)
CDs.reset_index(inplace=True, drop=True)

In [ ]:
# URL para calcular distância na Google Cloud

def calculando_distancias(origem, destino):
    url = "https://maps.googleapis.com/maps/api/distancematrix/json?"

    # Fazer solicitação HTTP
    r = requests.get(url + "origins="+origem+"&destinations="+destino+"&key="+api_key)

    # Extrair valores de distância e tempo de percurso da resposta
    distancia_valor = r.json()['rows'][0]["elements"][0]["distance"]["value"]
    distancia_texto = r.json()['rows'][0]["elements"][0]["distance"]["text"]
    tempo_valor = r.json()['rows'][0]["elements"][0]["duration"]["value"]
    tempo_texto = r.json()['rows'][0]["elements"][0]["duration"]["text"]

    return distancia_valor

In [ ]:
# Definindo matrizes de distancias

#clusters = np.unique(rotulos_clusters)

#for cluster in clusters:
base = CDs[CDs['CLUSTERS'] == 1]
base.reset_index(inplace=True, drop=True)

base2 = base.drop(columns=['LAT','LNG','CLUSTERS'])

In [ ]:
origens = base['CITY - UF']

for origem in origens:
    base2[origem] = origem

In [ ]:
origens = pd.Series(base['CITY - UF'])
destinos = origens

for linha in range(0,base2.shape[0]):
    origem = base2.loc[linha,'CITY - UF']
    for coluna in range(3,base2.shape[1]):
        destino = base2.iloc[linha,coluna]
        
        if origem == destino:
            base2.iloc[linha,coluna] = 0          
        else:
            base2.iloc[linha,coluna] = calculando_distancias(origem,destino)

In [ ]:
base2